In [118]:
# If you what use ngrams.py you should use python2
# Or, otherwise, you need to modify ngrams.py by yourself in order to use it in python3.
from __future__ import print_function
from __future__ import unicode_literals
from string import punctuation
import re
import numpy as np
from ngrams import ngrams
from collections import defaultdict
from bitweight import BitWeight, BitWeightRangeError

In [119]:

small_corpus = ['Why dont we start here',
                  'Why dont we end there',
                  'Let us start with a few other examples',
                  'We never start with an example with so few tokens',
                  'Tokens can be words that we start with in example docs']


In [120]:
# TOKENIZE - converts lists of sentences, like in the small_corpus
# to list of list of tokens. All of the other functions will require
# their parameters in the form of the outupt of the tokenize function.
def tokenize(corpus):
    tokens = [sentence.split(' ') for sentence in corpus]
    return tokens



# HW3: Language Modeling
For this part of the assignment, you will implement two simple count-based n-gram language models: one based on maximum-likelihood estimation, and another based on Witten-Bell smoothing. The data you will be using is a subset of the Penn Treebank's tagged Wall Street Journal articles on which we have done some initial processing. There are two versions of the data for this assignment:

##### wsj.pos.gz
##### wsj-normalized.pos.gz
The difference is that, in the second (normalized) version of the data, we have collapsed some entries from certain tag categories (e.g. CDs, NNPs, etc.) into type-tokens to help reduce sparsity. Take a look at the data and see for yourself. Consider: what would be the benefits and drawbacks to this method of sparsity reduction? Note that, for this part of the assignment, the tags are un-necessary, so you'll want to work with the un-normalized version of the corpus.

### Task 1: produce a tag-free corpus

For this task, you have two jobs. 
* First, you need to write a function to filter out all tags. 
* Second, Make sure your code works for both wsj.pos.gz and wsj-normalized.pos.gz

####What to turn in
* your code
* some samples to show me that your code works as it should be

In [121]:
# FILE_TO_LIST - turns the wsj files into the form of the 'small_corpus'
# to prepare it to be the input parameter of the 'tokenize()' function.
def file_to_list(filename):
    with open(filename, 'r') as content_file:
        content = content_file.read()
        no_tags = re.sub('(/[A-Z$]{2,4})|(\.\s+[a-z])|(/[.,$])|[\\,/\'`]', '', content)
        return re.split('[\n]',no_tags)
        

<font color="red">Self assessment:</font>

### Maximum Likelihood
Now, start by producing code to compute maximum-likelihood estimate probabilities. Your code should be configurable with respect to the n-gram order- i.e., you should be able to set it to compute bigram, trigram, 4-gram, etc. probabilities. Refer to J&M and the lecture slides for definitions as needed. If you would like to write your own n-gram tokenization code, feel free to do so, but you may also use the ngrams.py utility class which contains a routine to take a list of tokens and produce a stream of n-grams with appropriate padding for the start and end of sentences.

#### Tip: 
* Start with a very small "toy" corpus of just a couple of sentences for debugging. 

* As discussed in class, I strongly recommend using nested defaultdicts as the foundational data structure for your language model, where the "outer" key is the prefix, and the value retrieved by that prefix is a second defaultdict  containing possible suffices for that prefix, each of which is an "inner" key. E.g., p("TRUTHS" | "HOLD THESE") would be retrieved by first looking up "HOLD THESE" and then from the resulting dictionary, looking up "TRUTHS": prob = trigrams[("HOLD","THESE")]["TRUTHS"] . Note that this arrangement makes it very easy to e.g. find out the number of times a given history occurs, the total probability mass assigned to all of a history's continuations, etc., all of which will be extremely helpful in the next part of the assignment.

* Use tuples to represent prefixes. E.g., instead of the string "HOLD THESE", use the tuple ("HOLD", "THESE"). Note that, in Python, lists are mutable, and therefore may not be used as keys in dictionaries- but tuples are immutable, and so make excellent keys.

* Don't forget about numerical underflow issues! You'll want to represent probabilities as negative base-2 log probabilities, and modify your arithmetic accordingly. I recommend experimenting with [the bitweight Python library](https://github.com/stevenbedrick/bitweight) (see its unit tests for example usage).
* 

#### What to turn in:
* your code 
* use your code to create a simple language model for small_corpus named as small_lm and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create language model for wsj.pos.gz named as wsj_lm

In [122]:

# COUNT_BUILDER - generates count models where 'corpus' is in 
# the form output by 'tokenize()'. Order is the 'n' in n-gram.
def count_builder(corpus, order):
    
    #ngram
    ng = ngrams(corpus, order)

    # describe model datatype
    model = defaultdict(lambda: defaultdict(int))
    
    # loop to build embedded defaultdict    
    for gram in ng: 
        if not gram[1] in model[gram[0]]:
            model[gram[0]][gram[1]] = 1
        else:
            model[gram[0]][gram[1]] += 1

    # Count Model
    return model



# MAX_LIKELIHOOD - converts a count model into its MLE form.
# 'count_model' is in the form output by 'count_builder()'
def max_likelihood(count_model):
    
    # Container for MLE model with BitWeight probabilities
    # Returns 0 for unseen values.
    model = defaultdict(lambda: defaultdict(lambda: BitWeight(0)))
    
    # for prefixes in model...
    for prefix, suffix_dict in count_model.iteritems():
        w_minus = BitWeight(0) # used to count total tokens
        
        # for words with hist prefix ...
        for suffix, count in suffix_dict.iteritems():
            w_minus += BitWeight(count) # add to total number of tokens
        
        # again, for words with hist prefix ...
        for suffix, count in suffix_dict.iteritems():
            model[prefix][suffix] = BitWeight.__itruediv__(BitWeight(count),w_minus) # set output probabilities
    
    # MLE Probabilities
    return model


<font color="red">Self assessment:</font>

### Smoothing

Once you’ve got an unsmoothed model working, move on to implementing Witten-Bell smoothing. Refer to the slides and J&M for details on how that ought to work.

#### Tip: 
* You can modify an already-populated defaultdict to change its default value (for example, to store a default backoff value for a particular history) by changing the object’s default_factory attribute. Consult the documentation for examples of how this works.
* As defined, W-B smoothing is highly recursive; you may find it more efficient to re-frame the algorithm in iterative terms.
* As in the previous section, start small.
* [This may offer you some help on how to implement Witten-Bell smoothing](http://www.ee.columbia.edu/~stanchen/e6884/labs/lab3/x207.html)


#### What to turn in:
* your code 
* use your code to create a simple smoothed language model based on small_lm  and show me that your output is correct(This is a small coupus so you could manully calculate the probalility).
* use your code to create a smoothed language model based on wsj_lm

In [124]:

def wb_model_builder(tokens, order):
    
    # build count model and MLE model to calculate WB values
    count_mod = count_builder(tokens, order)
    mle_mod = max_likelihood(count_mod)
    
    
    # Base Case: Unigram Model Builder
    if order == 1:
        
        # get tokens and types to calc lmbda(.)
        tks = sum(count_mod[()].values())
        tps = len(count_mod[()].keys())
        
        # use above to build lambda(.)
        lambda_dot = BitWeight(tks,(tks+tps))
        one_minus_lambda_dot = BitWeight(tps,(tks+tps))
        one_over_V = BitWeight(1,tps)
        
        # create return for unseen values Pwb(.) = (1 - lambda) * (1/V)
        default_return = one_minus_lambda_dot*one_over_V

        # declare WB unigram model to be handed back
        wb_model = defaultdict(lambda: defaultdict(lambda: default_return))
        
        # build WB unigram model
        for prefix, suffix_dict in mle_mod.iteritems() :
            for suffix, mle_prob in suffix_dict.iteritems():
                
                # Set WB Probability: Pwb(w) = lambda(.)*Pmle + one_minus_lambda(.)*one_over_V
                wb_model[prefix][suffix] = (lambda_dot * mle_prob) + (one_minus_lambda_dot*one_over_V)
        
        # return WB unigram model
        return wb_model
    
    
    # Recursive Case: If the order is greater than 1
    else:

        # get the model of P(w|h_n-1)
        wb_minus_one_model = wb_model_builder(tokens, order-1)

        # retrieve default value from lower order model
        default_return = wb_minus_one_model["x"]["x"] # xx will not be in any model

        # declare WB model with inhereted default
        wb_model = defaultdict(lambda: defaultdict(lambda: default_return))

        # for each history... 
        for prefix, suffix_dict in count_mod.iteritems():

            # calculate history parameters
            c_hist = sum(suffix_dict.values())
            N_one_plus = len(suffix_dict.keys())

            # build lambda terms
            lambda_hist = BitWeight(c_hist,(c_hist+N_one_plus))
            one_minus_lambda_hist = BitWeight(N_one_plus,(c_hist+N_one_plus))

            # set default returns for inner dictionary
            wb_model[prefix] = defaultdict(lambda: one_minus_lambda_hist*wb_minus_one_model[prefix[1:]]['xxxxx'])

            # for each word that follows that history ... 
            for suffix, wb_minus_prob in mle_mod[prefix].iteritems():

                # Set WB Probability: Pwb(w|hist) = lambda_hist*Pmle(w|hist) + one_minus_lambda_hist*Pwb(w|hist-1)
                lambda_hist_x_mle = lambda_hist*mle_mod[prefix][suffix]
                one_minus_lambda_x_wb_minus = one_minus_lambda_hist*wb_minus_one_model[prefix[1:]][suffix]
                wb_model[prefix][suffix] = lambda_hist_x_mle + one_minus_lambda_x_wb_minus
    
    # return WB ngram model
    return wb_model


<font color="red">Self assessment:</font>

### Evaluation via Perplexity
Explore the effects of n-gram order using perplexity. Perform ten-fold cross-validation on the WSJ corpus. On each iteration, this will give you a different 90/10 training/test split; train a smoothed language model on the 9 training sections, and compute the average per-token perplexity of the tenth section. The slides from the language modeling lecture give the equation for perplexity computation (as does J&M chapter 4); you'll need to modify the equation a bit, since we're using log-probabilities. 

Now, try this for unigram, bigram, trigram, and 4-gram models. 

#### What to turn in
* your cross-validation function. You are not suppose to use any cross-validation function from any module. You should implement it by yourself.
* your perplexity function
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj.pos.gz
* cross-validation result for unigram, bigram, trigram, and 4-gram models on wsj-normalized.pos.gz.
* Answer following 2 questions: 
    * How does perplexity change as the model order size increases?
    * How does perplexity change as the data changed?

In [159]:

#N_FOLD: generalized n_fold cross validation
#
#   # INPUTS
#   data: the dataset as a list
#   n: number of folds
#   scoring_function: function used to evaluate
#   param_list: list of parameters for scoring_function
#
#   # OUTPUTS
#   score_list: a list of the scores for the different test runs
def n_fold(data, n, scoring_function, param_list):
    
    # get lenght of corpus and use it to get test set size
    size_of_data = len(data)
    size_of_test_set = int(size_of_data/n)
    
    # score list holds onto results
    score_list = []
    
    ###  n folds  ##
    for x in range(n):
        
        # build test set and trainging set
        test_set = data[x*size_of_test_set:(x+1)*size_of_test_set]
        training_set = data[:x*size_of_test_set]+data[(x+1)*size_of_test_set:]        
        
        # evaluate
        score_list.append(scoring_function(test_set, training_set, *param_list))
          
    return score_list

# perplexity evaluation function
def wb_perplexity(test_set, training_set, order):
    
    N = 0.0
    
    # build witten_bell model from training data
    training_tokens = tokenize(training_set)
    wb_model = wb_model_builder(training_tokens, order)
    
    # get ngram of test set
    test_tokens = tokenize(test_set)
    test_grams = ngrams(test_tokens, order)

    # declare probability product holder
    p_w_all = BitWeight(1)
    
    # loop through all test data
    for history, word in test_grams:
        N += 1.0
        p_w_all *= wb_model[history][word]
    
  
    return (-1/N)*p_w_all.log()


In [160]:
tl = file_to_list('wsj.pos')
n_fold(tl,10, wb_perplexity, [1])

[-10.422096761477599,
 -10.45987762639243,
 -10.390216144723228,
 -10.401447688780395,
 -10.401991318443159,
 -10.383062018619954,
 -10.429966097216424,
 -10.33098178113066,
 -10.383009808610066,
 -10.458665963681328]

In [161]:
n_fold(tl,10, wb_perplexity, [2])

[-10.429471678720232,
 -10.569299379698139,
 -10.210676165552446,
 -10.255455726445804,
 -10.224240746015122,
 -10.116058976377726,
 -10.397244046784385,
 -10.116824865169251,
 -10.176933112809474,
 -10.313393760153046]

In [162]:
n_fold(tl,10, wb_perplexity, [3])

[-15.050208494884064,
 -15.139986528825176,
 -14.767778917626604,
 -14.742619734061412,
 -14.692135408294163,
 -14.620025683422407,
 -14.924272638298024,
 -14.628735285548975,
 -14.699512379465391,
 -14.787838561061625]

In [163]:
n_fold(tl,10, wb_perplexity, [4])

[-17.561782143002407,
 -17.57941885843227,
 -17.37648354408899,
 -17.358313582718672,
 -17.31948023526535,
 -17.312129805600605,
 -17.498584976944848,
 -17.325220475785834,
 -17.383820103722027,
 -17.356262932871307]

In [164]:
tln = file_to_list('wsj-normalized.pos')
n_fold(tln,10, wb_perplexity, [1])

[-9.139841449952604,
 -9.090189144811731,
 -8.994569726520528,
 -9.00566048016694,
 -8.99029134688309,
 -8.934111762343461,
 -8.976347463234887,
 -8.907302152696895,
 -8.991991926757937,
 -8.992503727615507]

In [165]:
n_fold(tln,10, wb_perplexity, [2])

[-8.886379038978031,
 -8.871260685136237,
 -8.556239133816296,
 -8.659173421699325,
 -8.580873334142375,
 -8.40541043191283,
 -8.602130967327128,
 -8.403942713660749,
 -8.521394160644576,
 -8.585333997959253]

In [166]:
n_fold(tln,10, wb_perplexity, [3])

[-12.725209756920828,
 -12.61834251039997,
 -12.432612715120497,
 -12.327071449985883,
 -12.33476832164793,
 -12.179958747984426,
 -12.416473294111954,
 -12.248306302410391,
 -12.314861205078136,
 -12.275600221224876]

In [167]:
n_fold(tln,10, wb_perplexity, [4])

[-15.31460434529303,
 -15.215397812061918,
 -15.119107808367305,
 -15.031293781066399,
 -14.994357118905963,
 -14.851657548784262,
 -15.014035985130482,
 -14.95445233656557,
 -14.957400540425956,
 -14.833688024612025]

<font color="red">Self assessment:</font>